In [1]:
from gurobipy import *
myModel=Model("my_model_name")

Academic license - for non-commercial use only - expires 2022-09-10
Using license file C:\Users\guoyu\gurobi.lic


In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('street.csv')
length = len(data['Stop_id'])

In [4]:
# x_ij = 1 if station i to station j is selected (routing variable)
x = myModel.addVars([i for i in range(length)],[j for j in range(length)],vtype=GRB.BINARY, name="x")
# y_i = integer (number of students on bus after station i)
y = myModel.addVars([i for i in range(length)],vtype=GRB.INTEGER, name="y")
# z_i = continuous (time at station i)
z = myModel.addVars([i for i in range(length)], name="z")

In [5]:
c = data['Number']
Q = 130
M = 1000
D = 1000
def t(i,j):
    return np.linalg.norm(np.array(data['Latitude'][i],data['Longitude'][i])-np.array(data['Latitude'][j],data['Longitude'][j]))

In [6]:
#dist=[]
#for i in range(1,length):
#    dist.append(t(0,i))
#print(max(dist))
#print(min(dist))

In [7]:
# for every station j (except for the school) exactly one bus
for j in range(1,length):
    myModel.addConstr(quicksum(x[i,j] for i in range(length))==1, name="Constraints1")

for i in range(1,length):
    # for every station i go to a next station j (or school?)
    myModel.addConstr(quicksum(x[i,j] for j in range(length))==1, name="Constraints2")
    # after every station i satisfy capacity
    myModel.addConstr(y[i]<=Q)

In [8]:
for i in range(length):
    for j in range(1,length):
        myModel.addConstr(y[i]-y[j]<= M-M*x[i,j]-c[j], name="Constraints3")
        myModel.addConstr(z[i]+t(i,j)-z[j]<= M*(1-x[i,j]), name="Constraints4")
for i in range(length):        
        myModel.addConstr(z[i]<=D, name="Constraints5")

In [9]:
myModel.addConstr(z[0]==0)
myModel.addConstr(y[0]==0)

<gurobi.Constr *Awaiting Model Update*>

In [10]:
Obj = M* quicksum(x[0,j] for j in range(1, length))+quicksum(t(i,j)*x[i,j] for i in range(length) for j in range(1,length))
myModel.setObjective(Obj,GRB.MINIMIZE)
myModel.write("test.lp")

In [ ]:
myModel.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 44699 rows, 22499 columns and 176123 nonzeros
Model fingerprint: 0x1d191d10
Variable types: 149 continuous, 22350 integer (22201 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 62015.240000
Presolve removed 606 rows and 299 columns
Presolve time: 0.62s
Presolved: 44093 rows, 22200 columns, 174766 nonzeros
Variable types: 148 continuous, 22052 integer (21904 binary)

Deterministic concurrent LP optimizer: primal and dual simplex (primal and dual model)
Showing first log only...

Presolved: 44093 rows, 22200 columns, 174766 nonzeros

Concurrent spin time: 0.01s

Solved with dual simplex (primal model)

Root relaxation: objective 3.853921e-01, 627 iterations, 0.59 seconds

    Nodes 

In [ ]:
myModel.write('file.lp')

In [ ]:
route=[]
for i in x:
    if x[i].X==1:
        route.append(i)
        print(i,x[i].X)
print(route)

In [ ]:
starting=[]
for i in range(len(route)):
    if route[i][0]==0:
        starting.append(route[i])
print(starting)

In [ ]:
routes=[]
for i in range(len(starting)):
    routes.append([])
    routes[i].append(starting[i])
print(routes)

In [ ]:
for j in range(len(routes)):
    for i in range(len(route)):
        if route[i][0]==routes[j][-1][1]:
            routes[j].append(route[i])
print(routes)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10] ## set the size of all figures

In [ ]:
import math
#def euclidean(x,y): # instead of the Haversine distance <- We should be using Haversine, this is simply a proxy
#    return math.sqrt((x[0]-y[0])**2+(x[1]-y[1])**2)

def haversine(x,y): 
    lon1=x[0]
    lat1=x[1]
    lon2=y[0]
    lat2=y[1]
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    r = 3956 
    return c*r

In [ ]:
from itertools import combinations
x=data["Longitude"].tolist() # Longitude corresponds to the x axis.
y=data["Latitude"].tolist() # Latitude corresponds to the y axis.
z=data["Stop_id"].tolist() 
s=data['Nodesize'].tolist()

stations={}
pos={}
for i in range(len(x)):
    stations[z[i]]=(x[i],y[i]) # storing (x,y) information for each zip code
    pos[z[i]]=(x[i],y[i]) # also storing them to position the nodes in the end.

G=nx.Graph() 
addList=[]
for (i,j) in combinations(stations,2):
    distance=haversine(stations[i],stations[j])
    addList.append(distance)
    G.add_node(i,weight=distance)
    G.add_edge(i,j,weight=distance)

In [ ]:
node_list=[0]
nx.draw(G,pos, with_labels=False, node_size=s, width=0) 
nx.draw_networkx_nodes(G,pos,node_size=50,nodelist=node_list,node_color ='r')

nx.draw_networkx_edges(G,pos,edge_color='r',edgelist=routes[0])
#nx.draw_networkx_edges(G,pos,edge_color='b',edgelist=routes[1])
#nx.draw_networkx_edges(G,pos,edge_color='y',edgelist=routes[2])
#nx.draw_networkx_edges(G,pos,edge_color='g',edgelist=routes[3])
#nx.draw_networkx_edges(G,pos,edge_color='k',edgelist=routes[4])
#nx.draw_networkx_edges(G,pos,edge_color='c',edgelist=routes[5])
#nx.draw_networkx_edges(G,pos,edge_color='m',edgelist=routes[6])
nx.draw_networkx_edges(G,pos,edge_color='lime',edgelist=routes[7])


plt.show()